In [577]:
from binascii import hexlify
from binascii import unhexlify
from hashlib import sha256
from hashlib import sha512
import hmac
from ecc import PrivateKey, S256Point, Signature
from helper import encode_base58_checksum, decode_base58, decode_base58_extended, hash160
from io import BytesIO, StringIO

N = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141

In [578]:
def extended_privkey(depth,finger_print,_index,chain_code,privkey, testnet=False):
    if testnet:
        version= 0x04358394.to_bytes(4,"big")
    else:
        version= 0x0488ade4.to_bytes(4,"big")
    depth=depth.to_bytes(1,"big")
    fingerprint =finger_print
    index = _index.to_bytes(4,"big")
    return version+depth+fingerprint+index+chain_code+b'\x00'+privkey

def deserialize_xprvk(s):
    version = s.read(4)
    depth = s.read(1)
    fingerprint = s.read(4)
    index= s.read(4)
    chain_code = s.read(32)
    s.read(1)
    privkey = s.read(32)
    return [version,depth, fingerprint,index,chain_code,privkey]
"""
xtended_private_key = extended_privkey(master_priv_key, chain_code, 0)
xtended_private_key = encode_base58_checksum(xtended_private_key)
xtended_private_key
"""

'\nxtended_private_key = extended_privkey(master_priv_key, chain_code, 0)\nxtended_private_key = encode_base58_checksum(xtended_private_key)\nxtended_private_key\n'

In [579]:
xtprvk = 'xprv9s21ZrQH143K31xYSDQpPDxsXRTUcvj2iNHm5NUtrGiGG5e2DtALGdso3pGz6ssrdK4PFmM8NSpSBHNqPqm55Qn3LqFtT2emdEXVYsCzC2U'
decoded =deserialize_xprvk( BytesIO(decode_base58_extended(xtprvk)))
decoded

[b'\x04\x88\xad\xe4',
 b'\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'`I\x9f\x80\x1b\x89m\x83\x17\x9aCt\xae\xb7\x82*\xae\xac\xea\xa0\xdb\x1f\x85\xee>\x90LM\xef\xbd\x96\x89',
 b"K\x03\xd6\xfc4\x04U\xb3c\xf5\x10 \xad>\xcc\xa4\xf0\x85\x02\x80\xcfClp\xc7'\x92?m\xb4l>"]

In [580]:
for x in decoded:
    print(hex(int.from_bytes(x,"big")))

0x488ade4
0x0
0x0
0x0
0x60499f801b896d83179a4374aeb7822aaeaceaa0db1f85ee3e904c4defbd9689
0x4b03d6fc340455b363f51020ad3ecca4f0850280cf436c70c727923f6db46c3e


In [581]:
master_priv_key = decoded[-1]
chain_code = decoded[-2]

In [582]:
pub_key = PrivateKey(int.from_bytes(master_priv_key,"big")).point.sec()
finger_print = hash160(pub_key)[:4]
print(hex(int.from_bytes(finger_print,"big")))

0xbd16bee5


In [583]:
i=0

In [584]:
msg=pub_key + i.to_bytes(4,"big")
msg

b'\x03\xcb\xca\xa9\xc9\x8c\x87z&\x97}\x00\x82\\\x95j#\x8e\x8d\xdd\xfb\xd3"\xcc\xe4\xf7K\x0b[\xd6\xac\xe4\xa7\x00\x00\x00\x00'

In [585]:
I= hmac.new(
            key = chain_code,
            msg=msg ,
            digestmod=sha512).digest()


In [586]:
I_L, I_R = I[:32], I[32:]

In [587]:
child_privkey_int = (int.from_bytes(I_L,"big") + int.from_bytes(master_priv_key,"big"))%N
hex(child_privkey_int)

'0xabe74a98f6c7eabee0428f53798f0ab8aa1bd37873999041703c742f15ac7e1e'

In [588]:
child_privkey = PrivateKey(child_privkey_int).wif(compressed= True)

In [589]:
child_chain_code = I_R

In [590]:
child_privkey

'L2ysLrR6KMSAtx7uPqmYpoTeiRzydXBattRXjXz5GDFPrdfPzKbj'

In [591]:
hex(int.from_bytes(child_chain_code,"big"))

'0xf0909affaa7ee7abe5dd4e100598d4dc53cd709d5a5c2cac40e7412f232f7c9c'

In [592]:
hex(int.from_bytes(PrivateKey(child_privkey_int).point.sec(),"big"))

'0x2fc9e5af0ac8d9b3cecfe2a888e2117ba3d089d8585886c9c826b6b22a98d12ea'

In [593]:
xtended_private_key = extended_privkey(1,finger_print,0,child_chain_code,child_privkey_int.to_bytes(32,"big"))
xtended_private_key = encode_base58_checksum(xtended_private_key)
xtended_private_key

'xprv9vHkqa6EV4sPZHYqZznhT2NPtPCjKuDKGY38FBWLvgaDx45zo9WQRUT3dKYnjwih2yJD9mkrocEZXo1ex8G81dwSM1fwqWpWkeS3v86pgKt'

In [594]:
 b'\xbd\x16\xbe\xe5',

(b'\xbd\x16\xbe\xe5',)